In [ ]:
!apt-get update


Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,280 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [71.0 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Package

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [ ]:
!wget https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz


--2025-09-18 05:37:34--  https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.208.237, 135.181.214.104, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.208.237|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 730107476 (696M) [application/x-gzip]
Saving to: ‘hadoop-3.3.6.tar.gz’

hadoop-3.3.6.tar.gz 100%[===================>] 696.28M  15.3MB/s    in 45s     

2025-09-18 05:38:19 (15.6 MB/s) - ‘hadoop-3.3.6.tar.gz’ saved [730107476/730107476]



In [ ]:
!tar -xvzf hadoop-3.3.6.tar.gz


Streaming output truncated to the last 5000 lines.
hadoop-3.3.6/share/doc/hadoop/hadoop-project-dist/hadoop-hdfs-client/build/source/hadoop-hdfs-project/hadoop-hdfs-client/target/api/org/apache/hadoop/hdfs/web/resources/BufferSizeParam.html
hadoop-3.3.6/share/doc/hadoop/hadoop-project-dist/hadoop-hdfs-client/build/source/hadoop-hdfs-project/hadoop-hdfs-client/target/api/org/apache/hadoop/hdfs/web/resources/DeleteOpParam.Op.html
hadoop-3.3.6/share/doc/hadoop/hadoop-project-dist/hadoop-hdfs-client/build/source/hadoop-hdfs-project/hadoop-hdfs-client/target/api/org/apache/hadoop/hdfs/web/resources/PutOpParam.html
hadoop-3.3.6/share/doc/hadoop/hadoop-project-dist/hadoop-hdfs-client/build/source/hadoop-hdfs-project/hadoop-hdfs-client/target/api/org/apache/hadoop/hdfs/web/resources/RenameOptionSetParam.html
hadoop-3.3.6/share/doc/hadoop/hadoop-project-dist/hadoop-hdfs-client/build/source/hadoop-hdfs-project/hadoop-hdfs-client/target/api/org/apache/hadoop/hdfs/web/resources/XAttrEncodingParam.

In [ ]:
!mv hadoop-3.3.6 /usr/local/hadoop

In [ ]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/usr/local/hadoop"
os.environ["PATH"] += ":/usr/local/hadoop/bin:/usr/local/hadoop/sbin"

!echo "JAVA_HOME=$JAVA_HOME"
!echo "HADOOP_HOME=$HADOOP_HOME"
!hadoop version

JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64
HADOOP_HOME=/usr/local/hadoop
Hadoop 3.3.6
Source code repository https://github.com/apache/hadoop.git -r 1be78238728da9266a4f88195058f08fd012bf9c
Compiled by ubuntu on 2023-06-18T08:22Z
Compiled on platform linux-x86_64
Compiled with protoc 3.7.1
From source with checksum 5652179ad55f76cb287d9c633bb53bbd
This command was run using /usr/local/hadoop/share/hadoop/common/hadoop-common-3.3.6.jar


In [ ]:
mapper_code = """#!/usr/bin/env python3
import sys
for line in sys.stdin:
    try:
        line = line.strip()
        datetime, temp = line.split(",")
        date = datetime.split(" ")[0]
        print(f"{date}\\t{temp}")
    except:
        continue
"""
with open("mapper.py", "w") as f:
    f.write(mapper_code)

# reducer.py
reducer_code = """#!/usr/bin/env python3
import sys
current_date = None
temps = []
for line in sys.stdin:
    line = line.strip()
    if not line:
        continue
    date, temp = line.split("\\t")
    temp = float(temp)
    if current_date == date:
        temps.append(temp)
    else:
        if current_date:
            print(f"{current_date}\\tmax={max(temps)}\\tmin={min(temps)}")
        current_date = date
        temps = [temp]
if current_date:
    print(f"{current_date}\\tmax={max(temps)}\\tmin={min(temps)}")
"""
with open("reducer.py", "w") as f:
    f.write(reducer_code)

# Sample weather data
data = """2025-09-01 14:00,35
2025-09-01 15:00,33
2025-09-01 16:00,37
2025-09-02 14:00,32
2025-09-02 15:00,34
"""
with open("weather_data.txt", "w") as f:
    f.write(data)

!chmod +x mapper.py reducer.py

In [ ]:
# ==========================
# 4. Run Hadoop Streaming
# ==========================

# Make input/output dirs
!hdfs dfs -mkdir -p /user/root/input
!hdfs dfs -put -f weather_data.txt /user/root/input/

# Remove old output if exists
!hdfs dfs -rm -r -f /user/root/output

# Run streaming job
!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-*.jar \
    -input /user/root/input/weather_data.txt \
    -output /user/root/output \
    -mapper mapper.py \
    -reducer reducer.py \
    -file mapper.py \
    -file reducer.py

# ==========================
# 5. Show Results
# ==========================
!hdfs dfs -cat /user/root/output/part-00000


2025-09-18 05:41:19,297 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper.py, reducer.py] [] /tmp/streamjob5892338845309209380.jar tmpDir=null
2025-09-18 05:41:20,090 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-09-18 05:41:20,353 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-09-18 05:41:20,353 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2025-09-18 05:41:20,406 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2025-09-18 05:41:20,724 INFO mapred.FileInputFormat: Total input files to process : 1
2025-09-18 05:41:20,752 INFO mapreduce.JobSubmitter: number of splits:1
2025-09-18 05:41:21,146 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local723345612_0001
2025-09-18 05:41:21,146 INFO mapreduce.JobSubmitter: Executing with tokens: []
2025-09-18 05:41:21,595 INFO mapred.LocalDistributedCacheM